### Necessary Imports & Declarations

In [6]:
from typing import Optional, TypeVar, Type
from pydantic import BaseModel, Field

import instructor
from openai import OpenAI, AsyncOpenAI
from langchain.schema.runnable import RunnableLambda

T = TypeVar('T', bound=BaseModel)

### Function to create an instructor integrated `runnable`

> Supports OpenAI compatible API. [Ollama, vLLM, ...]

In [7]:
def create_instructor_lambda(
    model: str,
    response_model: Type[T],
    base_url: Optional[str] = None,
    api_key: Optional[str] = None,
    mode: Optional[instructor.Mode] = None,
    **kwargs
) -> RunnableLambda[str, T]: 
    """
    Create an instructor integrated langchain runnable.
    @param model: Model to use.
    @param response_model: pydantic class to format output.
    @param base_url: Base URL to get chat completions.
    @param api_key: API key to use for requesting base URL.
    @param mode: instructor mode to use. [default: JSON]
    @param kwargs: Extra kwargs to pass to create method.
    """
    mode = mode or instructor.Mode.JSON
    client = instructor.patch(
        OpenAI(base_url=base_url, api_key=api_key), mode=mode)
    aclient = instructor.patch(
        AsyncOpenAI(base_url=base_url, api_key=api_key), mode=mode)

    build_kwargs = lambda x: {
        'model': model,
        'response_model': response_model,
        'messages': [{'role': 'user', 'content': x}]
        **kwargs
    }
    
    def func(__prompt: str):
        return client.chat.completions.create(**build_kwargs(__prompt))
    
    async def afunc(__prompt: str):
        return await aclient.chat.completions.create(**build_kwargs(__prompt))

    return RunnableLambda(func=func, afunc=afunc)

### Usage

- Create a Response Model that represents how the final output should be.
- Create a instructor runnable 
- Invoke the runnable

In [8]:
class Character(BaseModel):
    name: str
    age: int
    fact: str = Field(..., description="A fact about the character")

In [9]:
llm = create_instructor_lambda(
    model='llama2', 
    response_model=Character, 
    base_url="http://localhost:11434/v1", # Ollama default URL
    api_key="ollama" # not required, you can leave it empty
)

In [10]:
llm.invoke('Tell me about Elon Musk.')

Character(name='Elon Musk', age=49, fact='He is a successful entrepreneur and business magnate known for his innovative ideas and visionary leadership in the fields of space exploration, electric vehicles, and renewable energy.')

In [11]:
await llm.ainvoke('Tell me about Ichigo Kurosaki.')

Character(name='Ichigo Kurosaki', age=16, fact='Ichigo is a highly skilled Soul Reaper and the main protagonist of the series.')